In [ ]:
# default_exp datos

# Datos

> Cargar y preparar datos de rezago social, pobreza, salud, y de infraestructura de México. 

In [ ]:
#hide
from nbdev.showdoc import *

# Bibliotecas

In [ ]:
# export
import os
import glob

from fastcore.test import *

import pandas as pd
import geopandas as gpd
from osgeo import gdal, ogr
from datetime import timedelta


## Utilería

### columnas_comunes

In [ ]:
# exports

def columnas_comunes(df_1, df_2):
    '''
    Obtiene la lista de columnas con el mismo nombre entre dos DataFrame
    '''
    return list(set(df_1.columns).intersection(df_2.columns))

### arregla_cvegeo

In [ ]:
#exports

def arregla_cvegeo(df, campo_cvegeo='cvegeo', extraer_ent=False):
    '''
    Revisa que los indetificadores de municipio (clave entidad + clave municipio) tengan 5 digitos, si un identificador
    tiene 4 le agrega un cero. Esta corrección asume que el problema surgió de leer los identificadores como enteros
    lo cual provoca que se descarten los ceros a la izquierda.
    '''
    df[campo_cvegeo] = df[campo_cvegeo].apply(lambda x: '0' + x if len(x) == 4 else x)
    
    if extraer_ent:
        df[f'ent_{campo_cvegeo}'] = df[campo_cvegeo].str[:2]
    return df

## carga_datos_covid19_MX

In [ ]:
# exports
DATA_DIR_COVID = 'datos/secretaria_salud/'

def carga_datos_covid19_MX(fecha='200601', resolver_claves='sustitucion', periodo='dia'):
    """
        Lee en un DataFrame el CSV con el reporte de casos de la Secretaría de Salud de México publicado en una fecha dada. Esta función
        también lee el diccionario de datos que acompaña a estas publicaciones para preparar algunos campos, en particular permite la funcionalidad
        de generar columnas binarias para datos con valores 'SI', 'No'.
        
        **Nota**: En esta versión la ruta esta y nombre de los archivos es fija. Asumimos que existe un directorio 'datos/secretaria_salud/'
        donde se encuentran todos los archivos.
        
        periodo: 'dia', 'historico'. Trae los datos de la fecha indicada o trae
        la serie de conjuntos de datos hasta la fecha indicada (por implementar).
        
        resolver_claves: 'sustitucion', 'agregar', 'si_no_binarias', 'solo_localidades'. Resuelve los valores del conjunto de datos usando el
        diccionario de datos y los catálogos. 'sustitucion' remplaza los valores en las columnas, 'agregar'
        crea nuevas columnas. 'si_no_binarias' cambia valores SI, NO, No Aplica, SE IGNORA, NO ESPECIFICADO por 1, 0, 0, 0, 0 respectivamente.
        
    """
    
    file = os.path.join(DATA_DIR_COVID, f'{fecha}COVID19MEXICO.csv')
    df = pd.read_csv(file, dtype=object, encoding='latin-1')
    
    # Hay un error y el campo OTRA_COMP es OTRAS_COMP según los descriptores
    df.rename(columns={'OTRA_COM': 'OTRAS_COM'}, inplace=True)
    
    # Asignar clave única a municpios
    df['MUNICIPIO_RES'] = df['ENTIDAD_RES'] + df['MUNICIPIO_RES']
    df['CLAVE_MUNICIPIO_RES'] = df['MUNICIPIO_RES']
    
    # Resolver códigos de entidad federal
    entidades = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Catalogos_0412.xlsx',
                              'Catálogo de ENTIDADES',
                              dtype=str, encoding='latin-1')
    
    cols_entidad = ['ENTIDAD_RES', 'ENTIDAD_UM', 'ENTIDAD_NAC']
    df[cols_entidad] = df[cols_entidad].replace(to_replace=entidades['CLAVE_ENTIDAD'].values,
                                               value=entidades['ENTIDAD_FEDERATIVA'].values)

    # Leer diccionario y catálogo de datos
    municipios = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Catalogos_0412.xlsx',
                               'Catálogo MUNICIPIOS',
                               dtype=str)
    municipios['CLAVE_MUNICIPIO'] = municipios['CLAVE_ENTIDAD'] + municipios['CLAVE_MUNICIPIO']

    # Resolver códigos de municipio
    df['MUNICIPIO_RES'] = df['MUNICIPIO_RES'].replace(to_replace=municipios['CLAVE_MUNICIPIO'].values,
                                               value=municipios['MUNICIPIO'].values)
        
    # Resolver resultados
    tipo_resultado = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Catalogos_0412.xlsx',
                                   'Catálogo RESULTADO',
                                   dtype=str)
    df['RESULTADO'] = df['RESULTADO'].replace(to_replace=tipo_resultado['CLAVE'].values,
                                                              value=tipo_resultado['DESCRIPCIÓN'].values)
    
    # Resolver datos SI - NO

    # Necesitamos encontrar todos los campos que tienen este tipo de dato y eso 
    # viene en los descriptores, en el campo FORMATO_O_FUENTE
    
    descriptores = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Descriptores_0419.xlsx',
                                 index_col='Nº')
    descriptores.columns = list(map(lambda col: col.replace(' ', '_'), descriptores.columns))
    descriptores['FORMATO_O_FUENTE'] = descriptores.FORMATO_O_FUENTE.str.strip()

    datos_si_no = descriptores.query('FORMATO_O_FUENTE == "CATÁLOGO: SI_ NO"')
    cat_si_no = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Catalogos_0412.xlsx',
                              'Catálogo SI_NO',
                              dtype=str)
    cat_si_no['DESCRIPCIÓN'] = cat_si_no['DESCRIPCIÓN'].str.strip()
    
    campos_si_no = datos_si_no.NOMBRE_DE_VARIABLE
    nuevos_campos_si_no = campos_si_no
    
    if resolver_claves == 'agregar':
        nuevos_campos_si_no = [nombre_var + '_NOM' for nombre_var in campos_si_no]
    elif resolver_claves == 'si_no_binarias':
        nuevos_campos_si_no = [nombre_var + '_BIN' for nombre_var in campos_si_no]
        cat_si_no['DESCRIPCIÓN'] = list(map(lambda val: 1 if val == 'SI' else 0, cat_si_no['DESCRIPCIÓN']))
        
    
    df[nuevos_campos_si_no] = df[datos_si_no.NOMBRE_DE_VARIABLE].replace(
                                                to_replace=cat_si_no['CLAVE'].values,
                                                value=cat_si_no['DESCRIPCIÓN'].values)
    # Resolver tipos de paciente
    cat_tipo_pac = pd.read_excel('datos/secretaria_salud/diccionario_datos_covid19/Catalogos_0412.xlsx',
                                 'Catálogo TIPO_PACIENTE',
                                 dtype=str)
    df['TIPO_PACIENTE'] = df['TIPO_PACIENTE'].replace(to_replace=cat_tipo_pac['CLAVE'].values,
                                                      value=cat_tipo_pac['DESCRIPCIÓN'].values)
    
    df['FECHA_INGRESO'] = pd.to_datetime(df['FECHA_INGRESO'])
    df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'])
    df['FECHA_DEF'] = pd.to_datetime(df['FECHA_DEF'], 'coerce')
    df['DEFUNCION'] = (df['FECHA_DEF'].notna()).astype(int)
    df['EDAD'] = df['EDAD'].astype(int)
    
    df.set_index('FECHA_INGRESO', inplace=True)
    df['AÑO_INGRESO'] = df.index.year
    df['MES_INGRESO'] = df.index.month
    df['DIA_SEMANA_INGRESO'] = df.index.weekday
    df['SEMANA_AÑO_INGRESO'] = df.index.weekofyear
    df['DIA_MES_INGRESO'] = df.index.day
    df['DIA_AÑO_INGRESO'] = df.index.dayofyear
        
    return df

In [ ]:
casos_df = carga_datos_covid19_MX(resolver_claves='si_no_binarias')

In [ ]:
casos_df

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO_BIN,MIGRANTE_BIN,UCI_BIN,DEFUNCION,AÑO_INGRESO,MES_INGRESO,DIA_SEMANA_INGRESO,SEMANA_AÑO_INGRESO,DIA_MES_INGRESO,DIA_AÑO_INGRESO
FECHA_INGRESO,,,,,,,,,,,,,,,,,,,,,
2020-05-11,2020-06-01,04f3dd,2,3,SINALOA,2,SINALOA,SINALOA,CULIACÁN,AMBULATORIO,...,1,0,0,0,2020,5,0,20,11,132
2020-05-13,2020-06-01,1607b3,2,3,TABASCO,2,TABASCO,TABASCO,CENTRO,AMBULATORIO,...,1,0,0,0,2020,5,2,20,13,134
2020-05-22,2020-06-01,1b7c4b,2,3,TABASCO,1,TABASCO,TABASCO,COMALCALCO,HOSPITALIZADO,...,0,0,0,0,2020,5,4,21,22,143
2020-04-17,2020-06-01,03f6dd,2,4,CIUDAD DE MÉXICO,1,CIUDAD DE MÉXICO,MÉXICO,NEZAHUALCÓYOTL,AMBULATORIO,...,0,0,0,0,2020,4,4,16,17,108
2020-04-18,2020-06-01,098a35,2,4,BAJA CALIFORNIA,2,SINALOA,BAJA CALIFORNIA,TIJUANA,HOSPITALIZADO,...,0,0,1,1,2020,4,5,16,18,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-01,2020-06-01,11cd21,1,12,VERACRUZ DE IGNACIO DE LA LLAVE,2,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ,HOSPITALIZADO,...,0,0,0,0,2020,6,0,23,1,153
2020-05-30,2020-06-01,0527d5,1,12,CHIAPAS,2,CHIAPAS,CHIAPAS,SAN CRISTÓBAL DE LAS CASAS,HOSPITALIZADO,...,0,0,0,0,2020,5,5,22,30,151
2020-05-20,2020-06-01,101d66,2,11,CIUDAD DE MÉXICO,2,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,GUSTAVO A. MADERO,AMBULATORIO,...,0,0,0,0,2020,5,2,21,20,141


## agrupa_casos_municipios

In [ ]:
# export

def agrupa_casos_municipios(casos_df):
    # cuenta número de casos por resultado
    cols_localidad = ['ENTIDAD_RES', 'MUNICIPIO_RES', 'CLAVE_MUNICIPIO_RES']
    
    covid_municipal = casos_df[cols_localidad + ['RESULTADO', 'DEFUNCION']].groupby(cols_localidad + ['RESULTADO'])
    covid_municipal = covid_municipal.agg({'RESULTADO':'count', 'DEFUNCION':sum})    
    covid_municipal.rename(columns={'RESULTADO': 'conteo', 'DEFUNCION': 'defunciones'}, level=0, inplace=True)
    covid_municipal.reset_index(inplace=True)
    
    poblacion_2020 = pd.read_csv('datos/municipios/proyeccion_poblacion_2020.csv',
                                 dtype={'cve_ent': object, 'cve_mun': object, 'pob2020': int})
    covid_municipal = covid_municipal.merge(poblacion_2020, left_on='CLAVE_MUNICIPIO_RES', right_on='cve_mun')
    covid_municipal.rename(columns={'cve_ent': 'CLAVE_ENTIDAD_RES'}, inplace=True)
    covid_municipal.drop(columns='cve_mun', inplace=True)
    
    return covid_municipal


In [ ]:
casos_municipales = agrupa_casos_municipios(casos_df)
casos_municipales

,ENTIDAD_RES,MUNICIPIO_RES,CLAVE_MUNICIPIO_RES,RESULTADO,conteo,defunciones,CLAVE_ENTIDAD_RES,pob2020
0,AGUASCALIENTES,AGUASCALIENTES,01001,No positivo SARS-CoV-2,2756,11,01,961977
1,AGUASCALIENTES,AGUASCALIENTES,01001,Positivo SARS-CoV-2,652,30,01,961977
2,AGUASCALIENTES,AGUASCALIENTES,01001,Resultado pendiente,160,0,01,961977
3,AGUASCALIENTES,ASIENTOS,01002,No positivo SARS-CoV-2,32,0,01,50864
4,AGUASCALIENTES,ASIENTOS,01002,Positivo SARS-CoV-2,8,0,01,50864
...,...,...,...,...,...,...,...,...
4509,ZACATECAS,VILLANUEVA,32055,No positivo SARS-CoV-2,23,2,32,31804
4510,ZACATECAS,VILLANUEVA,32055,Positivo SARS-CoV-2,2,0,32,31804
4511,ZACATECAS,ZACATECAS,32056,No positivo SARS-CoV-2,337,7,32,155533
4512,ZACATECAS,ZACATECAS,32056,Positivo SARS-CoV-2,60,4,32,155533
